# Examples on how to use the new depth estimation functions

This demonstrates the Segmented Rolling Gradient method.

Initialise notebook

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import pprint

#====================================================================
# Import the main functions from the uos_depth_est_core.py module
#====================================================================

from uos_depth_est_core import (
    get_setitec_signals,
    depth_est_persegment_stats,
    depth_est_xls_persegment_stats,
    )

# Depth Estimation using the Segmented Rolling Gradient method

## Processing signals provided as arrays
A function is provided with torque and position array as input. 
* Returned value can be varied by the `simple_results` flag. 
* When `simple_results == True`, the function returns the keypoint locations as a list of floating point velues.
* When `simple_results == False`, the function returns a dictionary that contains the individual depths of the material, and the reference points of the entry/exit point estimations.

In [ ]:
file_path = r'data\\20110019_20110019_ST_490_3.xls'

: 

In [ ]:
pos, torque = get_setitec_signals(file_path)
depth_est_persegment_stats(torque, pos)

: 

The below uses the the XLS file reader version of the algorithm

In [ ]:
results = depth_est_xls_persegment_stats(file_path)

fig, ax = plt.subplots(figsize=(12, 8))
sns.lineplot(x=pos, y=torque, ax=ax, label='Torque vs Position', color='cyan')
ax.vlines(x=results, ymin=torque.min(), ymax=torque.max()*0.5, color='red', label="Key positions")
ax.set_xlabel("Position [mm]")
ax.set_ylabel("Torque [A]")
ax.set_title(f"Torque vs position, estimated depth {results[-1]-results[0]:.2f} mm")
for i, pos in enumerate(results):
    ax.text(pos, torque.max()*0.5, f"{i}", color='red')

: 

# Comparison of the depth estimation algorithm with the manual measurements

In [ ]:
file_demo = r'data\\17070143_17070143_ST_1092_1.xls'
results_ss = depth_est_xls_persegment_stats(file_demo, simple_results=False)
results_smp = dict(depth_estimate = 12.64-4.81, estimated_positions= [4.81, 8.57, 12.64])

# Plot the results
fig, ax = plt.subplots(figsize=(12, 8))
pos, torque = get_setitec_signals(file_demo)
sns.lineplot(x=pos, y=torque, ax=ax, label='Torque vs Position', color='cyan')
ax.vlines(x=results_ss['estimated_positions'], ymin=torque.max()*0.25, ymax=torque.max()*0.45, color='purple', label="Segmented Gradient (SG) key positions")
ax.vlines(x=results_smp['estimated_positions'], ymin=torque.max()*0.5, ymax=torque.max()*0.7, color='blue', label="Segmented MP (SMP) key positions")
ax.set_xlabel("Position [mm]")
ax.set_ylabel("Torque [A]")
ax.set_title(f"Torque vs position, reference coupon material: 4 mm CFRP + 4 mm Ti = 8 mm coupon thickness")
for i, pos in enumerate(results_ss['estimated_positions']):
    ax.text(pos, torque.max()*0.46, f"SG-{i}\n{results_ss['estimated_positions'][i]:.2f}", color='purple')
for i, pos in enumerate(results_smp['estimated_positions']):
    ax.text(pos, torque.max()*0.71, f"SMP-{i}\n{results_smp['estimated_positions'][i]:.2f}", color='blue')
ax.legend()

text = \
f"""Method 1 "Segmented Gradient (SG)":
  Total depth      =  {results_ss['depth_estimate']:.2f} mm
  SG-0 CFRP entry  =  {results_ss['estimated_positions'][0]:.2f} mm
  SG-1 Ti entry    =  {results_ss['estimated_positions'][1]:.2f} mm
  SG-2 Ti exit     = {results_ss['estimated_positions'][2]:.2f} mm
Method 2 "Segmented MP" (WIP):
  Total depth       =  {results_smp['depth_estimate']:.2f} mm
  SMP-0 CFRP entry  =  {results_smp['estimated_positions'][0]:.2f} mm
  SMP-1 Ti entry    =  {results_smp['estimated_positions'][1]:.2f} mm
  SMP-2 Ti exit     = {results_smp['estimated_positions'][2]:.2f} mm"""
ax.text(20, 10, text, color='black', bbox=dict(facecolor='green', alpha=0.5), fontdict=dict(size=10, family='monospace'))
7.85300016,
8.57499981, 12.63899994

: 